# Creating/Using Image Classifier


### Step 1 - Create Data for Classifier

In [1]:
import cv2

### Load HAAR Classifier


In [2]:
classifier = cv2.CascadeClassifier('cascade/haarcascade_frontalface_default.xml')

### Create a function to extract the face of the user

In [ ]:
def face_extractor(image):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(gray_image, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = image[y:y+h, x:x+w]

    return cropped_face

### Open video camera to take picture

In [ ]:
import os
from os import path

In [ ]:
name = input('Enter the name of the user for whom you are going to train the model')

In [ ]:
def checking_directory(name):
    file_name_path = 'faces/user/' + name
    if (not path.isdir(file_name_path)):
        try:
            os.mkdir(file_name_path)
            return True
        except:
            return False
    else:
        return True

In [ ]:
video = cv2.VideoCapture(0)
count = 0

while(True):
    ret, frame = video.read()
    face = frame
    if ((face_extractor(frame) is not None) & (checking_directory(name))):
        count = count + 1

        # Save file in specified directory with unique name
        file_name_path = 'faces/user/' + name + "/" + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), 
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Face Cropper', face)
    
    # the program will take 5 pictures of the user
    if cv2.waitKey(1) == 13 or count == 5: #13 is the Enter Key
        break
        
video.release()
cv2.destroyAllWindows()
if (count<3): 
    print("Unable to get user's face. Kindly run this code again.")
else:
    print("Face extraction complete. Now, start training the model.")

### Training our classifier for image recognition

In [12]:
import face_recognition

In [13]:
file_name_path = 'faces/user/' + '1.jpg'

In [14]:
user_image = face_recognition.load_image_file(file_name_path)

In [15]:
user_image_encoding = face_recognition.face_encodings(user_image)

In [16]:
user_face_name = ['Gurjas']
# user_face_name = name

In [23]:
import pickle

In [24]:
with open('trained_faces.dat', 'wb') as f:
    pickle.dump(user_image_encoding, f)

In [17]:
unknown_image = face_recognition.load_image_file('DSC_6160.jpg')

In [18]:
face_locations = face_recognition.face_locations(unknown_image)
face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

In [19]:
from PIL import Image, ImageDraw

In [20]:
pil_image = Image.fromarray(unknown_image)
# Create a Pillow ImageDraw Draw instance to draw with
draw = ImageDraw.Draw(pil_image)

In [21]:
# Loop through each face found in the unknown image
for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    # See if the face is a match for the known face(s)
    matches = face_recognition.compare_faces(user_image_encoding, 
                                             face_encoding)

    name = "Unknown"

    # If a match was found in known_face_encodings, just use the first one.
    if True in matches:
        first_match_index = matches.index(True)
        name = user_face_name[first_match_index]

    # Draw a box around the face using the Pillow module
    draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

    # Draw a label with a name below the face
    text_width, text_height = draw.textsize(name)
    draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
    draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))


# Remove the drawing library from memory as per the Pillow docs
del draw

# Display the resulting image
pil_image.show()

# You can also save a copy of the new image to disk if you want by uncommenting this line
pil_image.save("identified_images.jpg")